In [1]:
# !pip install mediapipe

In [2]:
import math
import cv2
# from google.colab.patches import cv2_imshow
import numpy as np
from time import time
import mediapipe as mp
import matplotlib.pyplot as plt

In [3]:
def calculateAngle(landmark1, landmark2, landmark3):
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2
    x3, y3, _ = landmark3

    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    
    if angle < 0:
        # angle += 360
        angle *= -1
    
    return angle

In [4]:
def ccw(A,B,C):

    a_x, a_y, _ = A
    b_x, b_y, _ = B
    c_x, c_y, _ = C

    return (c_y-a_y) * (b_x-a_x) > (b_y-a_y) * (c_x-a_x)

def isCrossingVectors(vec1_point1, vec1_point2, 
                      vec2_point1, vec2_point2):

    return ccw(vec1_point1,vec2_point1,vec2_point2) != ccw(vec1_point2,vec2_point1,vec2_point2) and \
              ccw(vec1_point1,vec1_point2,vec2_point1) != ccw(vec1_point1,vec1_point2,vec2_point2)

In [5]:
def getPoseAngle(landmarks):

  if landmarks:
    
    left_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])

    right_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])
 
    left_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                            landmarks[mp_pose.PoseLandmark.LEFT_HIP.value])

    right_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                              landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])

    left_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])

    right_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])
    
    right_hip_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value])
    
    left_hip_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                          landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                          landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value])

    return {'left_elbow_angle': left_elbow_angle,
            'right_elbow_angle': right_elbow_angle,
            'left_shoulder_angle': left_shoulder_angle,
            'right_shoulder_angle': right_shoulder_angle,
            'left_knee_angle': left_knee_angle,
            'right_knee_angle': right_knee_angle,
            'right_hip_angle': right_hip_angle,
            'left_hip_angle': left_hip_angle}

In [6]:
def getPoseCrossing(landmarks):

  if landmarks:

    crossing_forearm = isCrossingVectors(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])

    crossing_shin = isCrossingVectors(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])

    crossing_hip = isCrossingVectors(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value])

    crossing_ship_hip = isCrossingVectors(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]) or \
                        isCrossingVectors(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])

    return {'crossing_forearm': crossing_forearm,
            'crossing_shin': crossing_shin,
            'crossing_hip': crossing_hip,
            'crossing_ship_hip': crossing_ship_hip}

In [7]:
DESIRED_HEIGHT = 800
DESIRED_WIDTH = 800

def resize_and_show(image):
  h, w = image.shape[:2]
  if h < w:
    img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h/(w/DESIRED_WIDTH))))
  else:
    img = cv2.resize(image, (math.floor(w/(h/DESIRED_HEIGHT)), DESIRED_HEIGHT))
  # cv2_imshow(img)

# # Read images with OpenCV.
# images = {name: cv2.imread(name) for name in uploaded.keys()}
#
# for name, image in images.items():
#   resize_and_show(image)

In [8]:
# pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.3)
mp_pose_drawing = mp.solutions.drawing_utils

In [9]:
# hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.6)
mp_hands_drawing = mp.solutions.drawing_utils

In [10]:
def detectPose(image, pose, display2D=False, display3D=False):

    copy_image = image.copy()

    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(imageRGB)
    height, width, _ = image.shape
    landmarks = []

    if results.pose_landmarks:

        mp_pose_drawing.draw_landmarks(image=copy_image,
                                       landmark_list=results.pose_landmarks,
                                       connections=mp_pose.POSE_CONNECTIONS)

        for landmark in results.pose_landmarks.landmark:
            landmarks.append((int(landmark.x * width), int(landmark.y * height), (landmark.z * width)))

    if display2D:
        plt.figure(figsize=[22,22])
        plt.subplot(122);plt.imshow(copy_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');

    if display3D:
        mp_pose_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)

    return copy_image, landmarks

In [11]:
def detectHands(image, hands, display2D=False, display3D=False):

    copy_image = cv2.flip(image.copy(), 1)

    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(cv2.flip(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), 1))
    height, width, _ = image.shape
    landmarks = []

    result = {}

    i = 0

    # if results.multi_hand_landmarks:
    #
    #     for hand_landmarks in results.multi_hand_landmarks:
    #
    #     # hand_name = r_hands.multi_handedness[i].classification[0].label.lower()
    #     #
    #     # result[f'{hand_name}_'+'thumb_tip'] = { 'x': hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x * image_width,
    #     #                                         'y': hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y * image_hight }
    #     # result[f'{hand_name}_'+'index_finger_tip'] = { 'x': hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width,
    #     #                                                 'y': hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_hight }
    #     # result[f'{hand_name}_'+'middle_finger_tip'] = { 'x': hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x * image_width,
    #     #                                                 'y': hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y * image_hight }
    #     # result[f'{hand_name}_'+'ring_finger_tip'] = { 'x': hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].x * image_width,
    #     #                                               'y': hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y * image_hight}
    #     # result[f'{hand_name}_'+'pinky_tip'] = { 'x': hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].x * image_width,
    #     #                                         'y': hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].y * image_hight}
    #     # i += 1
    #
    #         mp_hands_drawing.draw_landmarks(image=copy_image,
    #                                         landmark_list=hand_landmarks,
    #                                         connections=mp_hands.HAND_CONNECTIONS)
    #
    #         for landmark in results.pose_landmarks.landmark:
    #             landmarks.append((int(landmark.x * width), int(landmark.y * height), (landmark.z * width)))


    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_hands_drawing.draw_landmarks(
            copy_image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS)

    if display2D:
        resize_and_show(cv2.flip(image, 1))

      # if display3D:
          # for name, image in images.items():
          #   if not r_hands.multi_hand_world_landmarks:
          #     continue
          #   for hand_world_landmarks in r_hands.multi_hand_world_landmarks:
          #     mp_hands_drawing.plot_landmarks(hand_world_landmarks, mp_hands.HAND_CONNECTIONS, azimuth=5)

    return cv2.flip(copy_image, 1)

In [12]:
def classifyPose(landmarks, output_image, display=False):

    label = 'Unknown Pose'
    color = (0, 0, 255)

    angels = getPoseAngle(landmarks)
    crossings = getPoseCrossing(landmarks)

    left_elbow_angle = angels['left_elbow_angle']
    right_elbow_angle = angels['right_elbow_angle']
    left_shoulder_angle = angels['left_shoulder_angle']
    right_shoulder_angle = angels['right_shoulder_angle']
    left_knee_angle = angels['left_knee_angle']
    right_knee_angle = angels['right_knee_angle']

    crossing_forearm = crossings['crossing_forearm']
    crossing_shin = crossings['crossing_shin']
    crossing_hip = crossings['crossing_hip']
    crossing_ship_hip = crossings['crossing_ship_hip']

    if crossing_forearm and \
            (left_elbow_angle > 25 and left_elbow_angle < 65) and \
            (right_elbow_angle > 25 and right_elbow_angle < 65) and \
            (left_shoulder_angle > 0 and left_shoulder_angle < 25) and \
            (right_shoulder_angle > 0 and right_shoulder_angle < 25):
        label = 'Negative'

    if label != 'Unknown Pose':
        color = (0, 255, 0)

    cv2.putText(output_image, label, (10, 30),cv2.FONT_HERSHEY_PLAIN, 2, color, 2)

    if display:
        plt.figure(figsize=[10,10])
        plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');

    else:
        return output_image, label

In [13]:
# from google.colab import files
#
# uploaded = files.upload()

In [14]:
# output_image, landmarks = detectPose(image, pose, False, False)
#
# angels = getPoseAngle(landmarks)
# crossings = getPoseCrossing(landmarks)
# res = getHandsData(output_image, hands, True, False)
#
# res, angels, crossings

In [15]:
# classifyPose(landmarks, output_image, display=True)

In [18]:
# Setup Pose function for video.
pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.7)
hands_video = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.6, max_num_hands=2)

# Initialize the VideoCapture object to read from the webcam.
camera_video = cv2.VideoCapture(0)
camera_video.set(3,1280)
camera_video.set(4,960)

# Initialize a resizable window.
cv2.namedWindow('Emotional state detector', cv2.WINDOW_NORMAL)

# Iterate until the webcam is accessed successfully.
while camera_video.isOpened():
    
    # Read a frame.
    ok, frame = camera_video.read()
    
    # Check if frame is not read properly.
    if not ok:
        
        # Continue to the next iteration to read the next frame and ignore the empty camera frame.
        continue
    
    # Flip the frame horizontally for natural (selfie-view) visualization.
    frame = cv2.flip(frame, 1)
    
    # Get the width and height of the frame
    frame_height, frame_width, _ =  frame.shape
    
    # Resize the frame while keeping the aspect ratio.
    frame = cv2.resize(frame, (int(frame_width * (640 / frame_height)), 640))
    
    # Perform Pose landmark detection.
    frame = detectHands(frame, hands_video)
    frame, landmarks = detectPose(frame, pose_video)

    # Check if the landmarks are detected.
    if landmarks:
        
        # Perform the Pose Classification.
        frame, _ = classifyPose(landmarks, frame)
    
    # Display the frame.
    cv2.imshow('Emotional state detector', frame)
    
    # Wait until a key is pressed.
    # Retreive the ASCII code of the key pressed
    k = cv2.waitKey(1) & 0xFF
    
    # Check if 'ESC' is pressed.
    if(k == 27):
        
        # Break the loop.
        break

# Release the VideoCapture object and close the windows.
camera_video.release()
cv2.destroyAllWindows()